In [1]:
import jax
jax.devices()

[CudaDevice(id=0)]

In [8]:
import openqdc
from openqdc.datasets import Spice as Splice
?Splice

Init signature:
Splice(
    energy_unit: Optional[str] = None,
    distance_unit: Optional[str] = None,
    array_format: str = 'numpy',
    energy_type: Optional[str] = 'formation',
    overwrite_local_cache: bool = False,
    cache_dir: Optional[str] = None,
    recompute_statistics: bool = False,
    transform: Optional[Callable] = None,
    skip_statistics: bool = False,
    read_as_zarr: bool = False,
    regressor_kwargs: Dict = {'solver_type': 'linear', 'sub_sample': None, 'stride': 1},
) -> None
Docstring:     
Spice dataset consists of 1.1 million conformations for a diverse set of 19k unique molecules consisting of
small molecules, dimers, dipeptides, and solvated amino acids. Conformations are first generated with RDKit,
and then molecular dynamics simulations at 100ps and 500K using OpenMM and Amber force field are used to generate
100 high energy conformations. Low-energy conformations are then generated by L-BFGS energy minimization and
molecular dynamics at 1ps and 100K.

In [9]:
from openqdc.utils.io import get_local_cache
print(get_local_cache())

/scicore/home/meuwly/boitti0000/.cache/openqdc


In [12]:

from openqdc.datasets import Spice

ds = Spice(
    energy_unit="kcal/mol",
    distance_unit="ang",
    cache_dir="/scicore/home/meuwly/boitti0000/spicecache",
    recompute_statistics=False,
    skip_statistics=True,
)

2025-11-05 12:59:27.835 | INFO     | openqdc.datasets.base:read_preprocess:435 - Reading preprocessed data.
2025-11-05 12:59:27.836 | INFO     | openqdc.datasets.base:read_preprocess:436 - Dataset spice with the following units:
                     Energy: hartree,
                     Distance: bohr,
                     Forces: hartree/bohr
2025-11-05 12:59:28.343 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded atomic_inputs with shape (33175288, 5), dtype float32
2025-11-05 12:59:28.343 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded position_idx_range with shape (1110165, 2), dtype int32
2025-11-05 12:59:28.344 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded energies with shape (1110165, 1), dtype float64
2025-11-05 12:59:28.344 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded forces with shape (33175288, 3, 1), dtype float32
2025-11-05 12:59:28.345 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded subset

In [48]:
not_charged = [ds[i]["charges"].sum() for i in range(len(ds))]
not_charged = np.array(not_charged)

In [49]:
not_charged.shape

(1110165,)

In [50]:
import numpy as np
where_not_charged = np.where(not_charged == 0)


AttributeError: 'tuple' object has no attribute 'shape'

In [51]:
where_not_charged[0].shape

(998243,)

In [110]:
ds.energy_methods

['wb97m-d3(bj)/def2-tzvppd']

In [111]:
from openqdc.methods import atom_energies

# method.atom_energies_dict

In [114]:
atom_energies = atom_energies.atom_energy_collection

In [116]:
import json

e0_dict = atom_energies # or whatever you extracted
# example: {'wb97m-d3(bj)/def2-tzvp': {('H',0): -0.5, ...}}

# JSON does not support tuple keys, so convert to strings
e0_json_ready = {}
for method, d in e0_dict.items():
    e0_json_ready[method] = {f"{elem}:{charge}": energy for (elem, charge), energy in d.items()}

# Save to file
with open("atomic_reference_energies.json", "w") as f:
    json.dump(e0_json_ready, f, indent=2)

In [104]:
ds[0].e0, ds[0].atomic_numbers

(array([[-23765.42563669],
        [-23765.42563669],
        [-23765.42563669],
        [-23765.42563669],
        [-23765.42563669],
        [-23765.42563669],
        [-23765.42563669],
        [-23765.42563669],
        [-33939.41501837],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ],
        [  -312.9767089 ]]),
 array([6, 6, 6, 6, 6, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1], dtype=int32))

In [122]:
import os, numpy as np

idx = where_not_charged[0]

# First pass: collect sizes
n_atoms = []
for i in idx:
    n_atoms.append(int(ds[i]["atomic_numbers"].shape[0]))
n_atoms = np.asarray(n_atoms, dtype=np.int32)
offsets = np.concatenate(([0], np.cumsum(n_atoms, dtype=np.int64)))
sumA = int(offsets[-1])
N = len(idx)

outdir = "openqdc_packed_memmap"
os.makedirs(outdir, exist_ok=True)


import numpy as np

def as_vec3(x: np.ndarray, name="array"):
    a = np.asarray(x)
    # common cases: (A,3) or (A,3,1)
    if a.ndim == 2 and a.shape[-1] == 3:
        return a.astype(np.float32, copy=False)
    if a.ndim == 3 and a.shape[-1] == 1 and a.shape[-2] == 3:
        return a[..., 0].astype(np.float32, copy=False)  # (A,3,1) -> (A,3)
    # safety: some datasets use (A,1,3)
    if a.ndim == 3 and a.shape[-2] == 1 and a.shape[-1] == 3:
        return a[:, 0, :].astype(np.float32, copy=False)  # (A,1,3) -> (A,3)
    raise ValueError(f"{name} has unsupported shape {a.shape}, expected (A,3) or (A,3,1)")

cursor = 0
for k, i in enumerate(where_not_charged[0]):
    item = ds[i]
    Z = np.asarray(item["atomic_numbers"], dtype=np.int32)
    R = as_vec3(item["positions"], "positions")
    F = as_vec3(item["forces"], "forces")
    a = Z.shape[0]

    # sanity checks
    if R.shape != (a, 3) or F.shape != (a, 3):
        raise ValueError(f"Mismatch: a={a}, R={R.shape}, F={F.shape}")

    Z_pack[cursor:cursor+a] = Z
    R_pack[cursor:cursor+a] = R
    F_pack[cursor:cursor+a] = F
    E[k]    = np.float64(item["formation_energies"])
    Qtot[k] = np.float64(item["charges"].sum())
    cursor += a

Z_pack.flush(); R_pack.flush(); F_pack.flush(); E.flush(); Qtot.flush()



/scratch/boitti0000/slurm-job.60808678/ipykernel_1904991/727650978.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  E[k]    = np.float64(item["formation_energies"])


In [132]:
import numpy as np
import jax
import jax.numpy as jnp
from typing import Iterator, Dict
import os
import os, numpy as np
import jax.numpy as jnp
import jax
from typing import Iterator, Dict

class PackedMemmapLoader:
    def __init__(self, path: str, batch_size: int, shuffle: bool=True, bucket_size: int=8192, seed: int=0):
        self.path = path
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.bucket_size = bucket_size
        self.rng = np.random.default_rng(seed)

        # Load metadata FIRST
        self.offsets = np.load(os.path.join(path, "offsets.npy"))
        self.n_atoms = np.load(os.path.join(path, "n_atoms.npy"))
        self.N = int(self.n_atoms.shape[0])
        sumA = int(self.offsets[-1])

        # Open read-only memmaps with explicit shapes
        self.Z_pack = np.memmap(os.path.join(path, "Z_pack.int32"), dtype=np.int32,  mode="r", shape=(sumA,))
        self.R_pack = np.memmap(os.path.join(path, "R_pack.f32"),  dtype=np.float32, mode="r", shape=(sumA, 3))
        self.F_pack = np.memmap(os.path.join(path, "F_pack.f32"),  dtype=np.float32, mode="r", shape=(sumA, 3))
        self.E      = np.memmap(os.path.join(path, "E.f64"),       dtype=np.float64, mode="r", shape=(self.N,))
        self.Qtot   = np.memmap(os.path.join(path, "Qtot.f64"),    dtype=np.float64, mode="r", shape=(self.N,))

        # Optional integrity checks
        def _expect_bytes(path, dtype, shape):
            return np.dtype(dtype).itemsize * int(np.prod(shape))
        def _filesize(path):
            return os.path.getsize(path)

        assert _filesize(os.path.join(path, "Z_pack.int32")) == _expect_bytes("Z_pack", np.int32,  (sumA,)),  "Z_pack size mismatch"
        assert _filesize(os.path.join(path, "R_pack.f32"))  == _expect_bytes("R_pack", np.float32, (sumA, 3)), "R_pack size mismatch"
        assert _filesize(os.path.join(path, "F_pack.f32"))  == _expect_bytes("F_pack", np.float32, (sumA, 3)), "F_pack size mismatch"
        assert _filesize(os.path.join(path, "E.f64"))       == _expect_bytes("E",      np.float64, (self.N,)), "E size mismatch"
        assert _filesize(os.path.join(path, "Qtot.f64"))    == _expect_bytes("Qtot",   np.float64, (self.N,)), "Qtot size mismatch"

        self.indices = np.arange(self.N, dtype=np.int64)

    def _yield_indices_bucketed(self) -> Iterator[np.ndarray]:
        order = self.indices.copy()
        if self.shuffle:
            self.rng.shuffle(order)
        for start in range(0, self.N, self.bucket_size):
            chunk = order[start:start+self.bucket_size]
            chunk = chunk[np.argsort(self.n_atoms[chunk], kind="mergesort")]
            for bstart in range(0, len(chunk), self.batch_size):
                yield chunk[bstart:bstart+self.batch_size]

    def _slice_mol(self, k: int):
        a0, a1 = int(self.offsets[k]), int(self.offsets[k+1])
        return (
            self.Z_pack[a0:a1],
            self.R_pack[a0:a1],
            self.F_pack[a0:a1],
            self.E[k],
            self.Qtot[k],
        )

    def batches(self) -> Iterator[Dict[str, jnp.ndarray]]:
        for batch_idx in self._yield_indices_bucketed():
            if len(batch_idx) == 0:
                continue
            Amax = int(self.n_atoms[batch_idx].max())
            B = len(batch_idx)

            Z = np.zeros((B, Amax), dtype=np.int32)
            R = np.zeros((B, Amax, 3), dtype=np.float32)
            F = np.zeros((B, Amax, 3), dtype=np.float32)
            M = np.zeros((B, Amax), dtype=bool)
            E = np.zeros((B,), dtype=np.float64)
            Qtot = np.zeros((B,), dtype=np.float64)

            for j, k in enumerate(batch_idx):
                z, r, f, e, q = self._slice_mol(int(k))
                a = z.shape[0]
                Z[j, :a] = z
                R[j, :a] = r
                F[j, :a] = f
                M[j, :a] = True
                E[j] = e
                Qtot[j] = q

            yield {
                "Z": jnp.array(Z),
                "R": jnp.array(R),
                "F": jnp.array(F),
                "mask": jnp.array(M),
                "E": jnp.array(E),
                "Qtot": jnp.array(Qtot),
            }


# usage
loader = PackedMemmapLoader("openqdc_packed_memmap", batch_size=128, shuffle=True, bucket_size=8192)
for i, batch in enumerate(loader.batches()):
    print(i, batch.keys())
    for k in batch.keys():
        print(k, batch[k].shape)

    if i > 10:
        break
    # batch["R"]: (B, Amax, 3), batch["mask"]: (B, Amax)
    # training step…



0 dict_keys(['Z', 'R', 'F', 'mask', 'E', 'Qtot'])
Z (128, 6)
R (128, 6, 3)
F (128, 6, 3)
mask (128, 6)
E (128,)
Qtot (128,)
1 dict_keys(['Z', 'R', 'F', 'mask', 'E', 'Qtot'])
Z (128, 7)
R (128, 7, 3)
F (128, 7, 3)
mask (128, 7)
E (128,)
Qtot (128,)
2 dict_keys(['Z', 'R', 'F', 'mask', 'E', 'Qtot'])
Z (128, 8)
R (128, 8, 3)
F (128, 8, 3)
mask (128, 8)
E (128,)
Qtot (128,)
3 dict_keys(['Z', 'R', 'F', 'mask', 'E', 'Qtot'])
Z (128, 9)
R (128, 9, 3)
F (128, 9, 3)
mask (128, 9)
E (128,)
Qtot (128,)
4 dict_keys(['Z', 'R', 'F', 'mask', 'E', 'Qtot'])
Z (128, 10)
R (128, 10, 3)
F (128, 10, 3)
mask (128, 10)
E (128,)
Qtot (128,)
5 dict_keys(['Z', 'R', 'F', 'mask', 'E', 'Qtot'])
Z (128, 11)
R (128, 11, 3)
F (128, 11, 3)
mask (128, 11)
E (128,)
Qtot (128,)
6 dict_keys(['Z', 'R', 'F', 'mask', 'E', 'Qtot'])
Z (128, 12)
R (128, 12, 3)
F (128, 12, 3)
mask (128, 12)
E (128,)
Qtot (128,)
7 dict_keys(['Z', 'R', 'F', 'mask', 'E', 'Qtot'])
Z (128, 13)
R (128, 13, 3)
F (128, 13, 3)
mask (128, 13)
E (128,)
Qtot

In [55]:
ds[0].keys()

dict_keys(['positions', 'atomic_numbers', 'charges', 'e0', 'energies', 'formation_energies', 'per_atom_formation_energies', 'name', 'subset', 'forces'])

In [98]:
ds[0].e0.sum(), ds[0].energies , ds[0].formation_energies, ds[0].energies - ds[0].formation_energies

(np.float64(-229696.4008720524),
 array([-232450.65274496]),
 array([-2754.2518729]),
 array([-229696.40087205]))

In [96]:
# ?Spice

In [24]:
cache_dir="/scicore/home/meuwly/boitti0000/spicecache"

In [25]:
from pathlib import Path

In [31]:
list(Path(cache_dir).glob("*/*/*"))

[PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/statistics/spice_forcescalculatorstats.pkl'),
 PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/statistics/spice_totalenergystats.pkl'),
 PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/statistics/spice_peratomformationenergystats_formation.pkl'),
 PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/statistics/spice_formationenergystats_formation.pkl'),
 PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/preprocessed/forces.mmap'),
 PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/preprocessed/position_idx_range.mmap'),
 PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/preprocessed/atomic_inputs.mmap'),
 PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/preprocessed/energies.mmap'),
 PosixPath('/scicore/home/meuwly/boitti0000/spicecache/spice/preprocessed/props.pkl')]

In [32]:
import pandas as pd
pd.read_pickle("/scicore/home/meuwly/boitti0000/spicecache/spice/preprocessed/props.pkl")

{'name': (array(['[Br-:1].[Br-:2]', '[Br-:2].[I-:1]',
         '[C+:4]([Cl:1])([Cl:2])[Cl:3]', ...,
         '[c:7]1([n:4][c:8]([n:6][c:9]([n:5]1)[Cl:3])[Cl:2])[Cl:1]',
         '[c:7]1([n:4][c:8]([n:6][c:9]([n:5]1)[F:3])[F:2])[F:1]',
         '[c:9]1([c:10]([c:12]([c:14]([c:13]([c:11]1[F:5])[F:7])[S:15](=[O:1])(=[O:2])[Cl:8])[F:6])[F:4])[F:3]'],
        shape=(19155,), dtype='<U632'),
  array([  182,   182,   182, ..., 16505, 16505, 16505], shape=(1110165,))),
 'subset': (array(['DES370K Dimers', 'DES370K Monomers', 'Dipeptides', 'Ion Pairs',
         'PubChem', 'Solvated Amino Acids'], dtype='<U20'),
  array([4, 4, 4, ..., 2, 2, 2], shape=(1110165,))),
 'n_atoms': array([27, 27, 27, ..., 44, 44, 44], shape=(1110165,), dtype=int32)}